In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [13]:
# 加载数据
raw_data = pd.read_excel("100clear.xls").head(13)  # 取前13条已评分数据
scores = pd.read_excel("scored_13.xlsx")      # 专家评分数据

In [14]:
# 确保列名一致
raw_data_columns = raw_data.columns.tolist()
for col in raw_data_columns:
    if "(" in col and ")" in col:
        indicator = col.split("(")[-1].split(")")[0]
        raw_data.rename(columns={col: indicator}, inplace=True)

In [15]:
# 特征工程：处理日期列
raw_data["C1"] = pd.to_datetime(raw_data["C1"])
raw_data["Year"] = raw_data["C1"].dt.year
raw_data["Month"] = raw_data["C1"].dt.month
raw_data.drop("C1", axis=1, inplace=True)  # 删除原始的日期列

In [16]:
# 定义定性特征和定量特征列名
qual_cols = ["C2", "C3", "C4", "C6", "C7", "C8", "C9", "C11", 
             "C13", "C14", "C15", "C16", "C17"]

In [17]:
# 确保这些列在数据中存在
available_columns = raw_data.columns.tolist()
qual_cols = [col for col in qual_cols if col in available_columns]

In [18]:
# 准备数据
X = raw_data[qual_cols]
y = scores  # 假设scores包含多个输出变量，且列名与X一致
# 拆分数据为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 特征预处理：处理缺失值并对特征进行编码
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                ('encoder', TargetEncoder())]), qual_cols)
    ])

In [19]:
from sklearn.preprocessing import OneHotEncoder  # 用于处理定性特征
# 定义特征处理器
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), qual_cols)  # 对类别特征进行独热编码
    ])

# 定义模型管道
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', MultiOutputRegressor(Ridge(alpha=1.0)))  # 使用岭回归模型
])

# 训练模型
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['C2', 'C3', 'C4', 'C6', 'C7',
                                                   'C8', 'C9', 'C11', 'C13',
                                                   'C14', 'C15', 'C16',
                                                   'C17'])])),
                ('model', MultiOutputRegressor(estimator=Ridge()))])

In [20]:
# 使用新数据进行预测
new_data = pd.read_excel("100clear.xls").iloc[13:]  # 获取新数据

# 处理新数据的列名
new_data_columns = new_data.columns.tolist()
for col in new_data_columns:
    if "(" in col and ")" in col:
        indicator = col.split("(")[-1].split(")")[0]
        new_data.rename(columns={col: indicator}, inplace=True)
# 处理新数据的日期列
new_data["C1"] = pd.to_datetime(new_data["C1"])
new_data["Year"] = new_data["C1"].dt.year
new_data["Month"] = new_data["C1"].dt.month
new_data.drop("C1", axis=1, inplace=True)  # 删除原始日期列
# 使用训练好的模型进行预测
predictions = pipeline.predict(new_data[qual_cols])
# 输出预测结果
print(predictions)

[[66.29080979 69.25407571 64.3770259  ... 72.92149795 67.83889529
  61.94728171]
 [71.29792574 74.10295558 69.07679116 ... 65.05527933 67.65694585
  62.68800914]
 [75.68466639 78.99110165 70.88340802 ... 73.6972618  75.76669872
  63.24467792]
 ...
 [70.60899676 69.664561   66.61588834 ... 73.24462561 70.42222826
  63.29290559]
 [74.6805427  72.09853645 66.6619055  ... 76.36589571 69.09685933
  63.75051184]
 [73.03489952 72.92925271 68.09696476 ... 65.68263308 71.96537282
  64.17915989]]


In [21]:
import pandas as pd
# 假设predictions是一个二维NumPy数组
predictions_df = pd.DataFrame(predictions)
# 显示DataFrame
print(predictions_df)

           0          1          2          3          4          5   \
0   66.290810  69.254076  64.377026  76.472820  68.423341  73.047327   
1   71.297926  74.102956  69.076791  67.937795  69.631294  81.989006   
2   75.684666  78.991102  70.883408  70.845268  71.621075  78.110467   
3   71.291970  74.930462  68.980244  72.972148  73.909190  71.561318   
4   70.917764  72.068777  65.242607  77.897753  74.033765  68.186888   
..        ...        ...        ...        ...        ...        ...   
82  66.130035  67.876639  65.689118  69.976517  67.886190  72.802979   
83  71.419302  69.657646  67.398855  68.816412  67.117347  77.448140   
84  70.608997  69.664561  66.615888  75.680403  69.404772  76.338856   
85  74.680543  72.098536  66.661905  79.446729  73.364194  72.929852   
86  73.034900  72.929253  68.096965  69.682889  70.284875  70.157466   

           6          7          8          9          10         11  \
0   60.292940  63.976700  75.886969  61.326980  67.023053  72.9

In [22]:
import pandas as pd
# 假设 predictions 是你模型的预测结果（一个二维 NumPy 数组）
predictions_df = pd.DataFrame(predictions)
# 将预测结果保存为 Excel 文件
predictions_df.to_excel("predictions100.xlsx", index=False, sheet_name="Predictions")